아래의 데이콘 BASELINE 코드 기본틀 토대로 수정하여 작성하였습니다.  
(https://dacon.io/competitions/official/235687/codeshare/2108?page=1&dtype=recent&ptype=pub)  

# 1. Library Import

In [104]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

RAW_PATH = '../data/raw/'
PATH = '../data/final/'

# 2. 데이터 불러오기 & 전처리

In [148]:
train_x = np.load(PATH + "train_df_err_cnt.npy")
test_x = np.load(PATH + "test_df_err_cnt.npy")

train_x_reg = pd.read_pickle(PATH  + "train_typecode_reg.pkl")
train_x_reg = np.reshape(train_x_reg.iloc[:,2:].values, [15000,-1])
test_x_reg = pd.read_pickle(PATH  + "test_typecode_reg.pkl")
test_x_reg = np.reshape(test_x_reg.iloc[:,2:].values, [14999,-1])

train_x_ratio = np.load(PATH + "train_ratio_unique_cnt_features.npy")
test_x_ratio = np.load(PATH + "test_ratio_unique_cnt_features.npy")
train_eq_tp_cd_prob = np.reshape(pd.read_pickle(PATH + "train_eq_tp_cd_prob.pkl").iloc[:,1].values, [15000,-1])

source_df = pd.DataFrame({"user_id" : [i for i in range(30000, 44999)]})
test_eq_tp_cd_prob = pd.merge(source_df, pd.read_pickle(PATH + "test_eq_tp_cd_prob.pkl"), on ="user_id", how ='left').fillna(0)

test_eq_tp_cd_prob = np.reshape(test_eq_tp_cd_prob.iloc[:,1].values, [14999,-1])
test_ty_cd_timestamp = pd.read_pickle(PATH + "test_ty_cd_timestamp_feature.pkl")
train_ty_cd_timestamp = pd.read_pickle(PATH + "train_ty_cd_timestamp_feature.pkl")
train_ty_cd_timestamp = np.reshape(train_ty_cd_timestamp.iloc[:,2:].values,[15000, -1])
test_ty_cd_timestamp = np.reshape(test_ty_cd_timestamp.iloc[:,2:].values,[14999, -1])


train_simul_prob_top_cnt = np.reshape(pd.read_pickle(PATH + "train_simul_prob_top_cnt.pkl").iloc[:,1].values, [15000, -1])
test_simul_prob_top_cnt = np.reshape(pd.read_pickle(PATH + "test_simul_prob_top_cnt.pkl").iloc[:,1].values, [14999,-1])

train_timedelta_static = np.load(PATH + "train_timedelta_static.npy")
test_timedelta_static = np.load(PATH + "test_timedelta_static_.npy")


fpca_results = pd.read_csv(PATH + "fpca_result_hours_rv.tsv", sep = '\t',  index_col=None)
train_fpca = fpca_results.iloc[:15000, 1:].values
test_fpca = fpca_results.iloc[15000:, 1:].values
train_quality_timestamp_features = pd.read_pickle(PATH + "train_quality_timestamp_features.pkl")
test_quality_timestamp_features = pd.read_pickle(PATH + "test_quality_timestamp_features.pkl")

use_cols =["count/time_min_max_intervalquality_"+str(i) for i in range(0,13)]
use_cols.remove('count/time_min_max_intervalquality_4')
use_cols.remove('count/time_min_max_intervalquality_3')

train_quality_timestamp_features = train_quality_timestamp_features[use_cols]
train_quality_timestamp_features = np.reshape(train_quality_timestamp_features.values, [15000, -1])

test_quality_timestamp_features = test_quality_timestamp_features[use_cols]
test_quality_timestamp_features =  np.reshape(test_quality_timestamp_features.values, [14999, -1])
train_quality_negaive_value_cnt = pd.read_pickle(PATH + "train_quality_negaive_value_cnt.pkl").fillna(0).iloc[:,1:].values
test_quality_negaive_value_cnt = pd.read_pickle(PATH + "test_quality_negaive_value_cnt.pkl").fillna(0).iloc[:,1:].values
train_simultaneous_quality_features = pd.read_pickle(PATH + "train_simultaneous_quality_features.pkl")
train_simultaneous_quality_features["ratio"] = train_simultaneous_quality_features["simul_cnt_max"]/train_simultaneous_quality_features["simul_cnt_mean"]
test_simultaneous_quality_features = pd.read_pickle(PATH + "test_simultaneous_quality_features.pkl")
test_simultaneous_quality_features["ratio"] = test_simultaneous_quality_features["simul_cnt_max"]/test_simultaneous_quality_features["simul_cnt_mean"]
train_simultaneous_quality_features = train_simultaneous_quality_features.fillna(0)
test_simultaneous_quality_features = test_simultaneous_quality_features.fillna(0)


print(train_x.shape)
train_x = np.append(train_x, train_x_reg, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_x_ratio, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_eq_tp_cd_prob, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_ty_cd_timestamp, axis =1) 
print(train_x.shape)
train_x = np.append(train_x, train_simul_prob_top_cnt, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_timedelta_static, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_fpca, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_quality_timestamp_features, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_quality_negaive_value_cnt, axis =1)
print(train_x.shape)
train_x = np.append(train_x, train_simultaneous_quality_features.iloc[:,1:].values, axis =1)
print(train_x.shape)


test_x = np.append(test_x, test_x_reg, axis = 1)
test_x = np.append(test_x, test_x_ratio, axis = 1)
test_x = np.append(test_x, np.reshape(test_eq_tp_cd_prob, [14999,1]), axis =1)
test_x = np.append(test_x, np.reshape(test_ty_cd_timestamp, [14999,259*6]), axis =1) # 원본 5 
test_x = np.append(test_x, test_simul_prob_top_cnt, axis =1)
test_x = np.append(test_x, test_timedelta_static, axis =1)
test_x = np.append(test_x, test_fpca, axis =1)
test_x = np.append(test_x, test_quality_timestamp_features, axis =1)
test_x = np.append(test_x, test_quality_negaive_value_cnt, axis =1)
test_x = np.append(test_x, test_simultaneous_quality_features.iloc[:,1:].values, axis =1)


(15000, 1322)
(15000, 1840)
(15000, 1893)
(15000, 1894)
(15000, 3448)
(15000, 3449)
(15000, 3613)
(15000, 3631)
(15000, 3994)
(15000, 4005)
(15000, 4011)


In [149]:
train_err_day_dummy_variable = pd.read_pickle(PATH + "train_err_day_dummy_variable.pkl")
test_err_day_dummy_variable = pd.read_pickle(PATH + "test_err_day_dummy_variable.pkl")
train_x = np.append(train_x, train_err_day_dummy_variable[["weekday_5_cnt","weekday_6_cnt","active_hour_ratio"]].values, axis =1)
test_x = np.append(test_x, test_err_day_dummy_variable[["weekday_5_cnt","weekday_6_cnt","active_hour_ratio"]].values, axis =1)

In [131]:
# import shap
# explainer = shap.TreeExplainer(model) # Tree model Shap Value 확인 객체 지정
# shap_values = explainer.shap_values(train_x) # Shap Values 계산
# shap_sum = np.abs(shap_values[1]).mean(axis=0)
# col_list = [i for i in range(4011)]
# importance_df = pd.DataFrame({"col_idx":col_list, "shap_val_mean":shap_sum})
# use_idx = importance_df.query("shap_val_mean>0").sort_values("shap_val_mean", ascending = False).col_idx

In [150]:
len(use_idx)

713

In [151]:
# 리더보드 재현을 위한 변수 조합 필터링(위의 전체 4011개로 학습한뒤 shap val mean 0인것은 제외하는 과정)
importance_df = pd.read_pickle(PATH + "importance_df.pkl")
use_idx = importance_df.query("shap_val_mean != 0 ").col_idx
train_x = train_x[:,use_idx]
test_x = test_x[:,use_idx]

In [152]:
train_prob = pd.read_csv(RAW_PATH+'train_problem_data.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련 
> 훈련 모델 파일이 미리 저장되어 있으므로 별도 트레인 없이 모델 json파일으로 predict 가능  

In [153]:
# 변수 이름 변경
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 665)
(15000,)


In [154]:
train_x = np.nan_to_num(train_x, 0)
test_x = np.nan_to_num(test_x, 0)

In [155]:

final_auc = []
final_auc_dic = {}
for random_state in [4]:
  # Train
  #-------------------------------------------------------------------------------------
  # validation auc score를 확인하기 위해 정의
  def f_pr_auc(probas_pred, y_true):
      labels=y_true.get_label()
      p, r, _ = precision_recall_curve(labels, probas_pred)
      score=auc(r,p) 
      return "pr_auc", score, True
  #-------------------------------------------------------------------------------------
  models     = []
  recalls    = []
  precisions = []
  auc_scores   = []
  threshold = 0.5
  # 파라미터 설정
  params =      {
                  'boosting_type' : 'dart',
                  'objective'     : 'binary',
                  'metric'        : 'auc',
                  }
  #-------------------------------------------------------------------------------------
  # 10-fold cross validation
  k_fold = KFold(n_splits=10, shuffle=True, random_state=random_state)
  from collections import Counter
  for train_idx, val_idx in k_fold.split(train_x,train_y):
      
      # split train, validation set
      X = train_x[train_idx]
      y = train_y[train_idx]
      valid_x = train_x[val_idx]
      valid_y = train_y[val_idx]
      d_train= lgb.Dataset(X, y)
      d_val  = lgb.Dataset(valid_x, valid_y)
      
      #run traning
      model = lgb.train(
                          
                          params,
                          train_set       = d_train,
                          num_boost_round = 300, 
                          valid_sets      = d_val,
                          feval           = f_pr_auc,
                          verbose_eval    = 50, 
#                           early_stopping_rounds = 100,
                        )
      
      # cal valid prediction
      valid_prob = model.predict(valid_x)
      valid_pred = np.where(valid_prob > threshold, 1, 0)
      
      # cal scores
      recall    = recall_score(valid_y, valid_pred)
      precision = precision_score( valid_y, valid_pred)
      auc_score = roc_auc_score(valid_y, valid_prob)

      # append scores
      models.append(model)
      recalls.append(recall)
      precisions.append(precision)
      auc_scores.append(auc_score)
  
      print('==========================================================')
  print(random_state, np.mean(auc_scores),np.std(auc_scores) )
  final_auc_dic[random_state] = np.mean(auc_scores) # 랜덤 state별 k-fold 별 auc score 
  final_auc.append(np.mean(auc_scores)) # 랜덤 state별 auc score 
  print(auc_scores)

[50]	valid_0's auc: 0.854929	valid_0's pr_auc: 0.78257
[100]	valid_0's auc: 0.856981	valid_0's pr_auc: 0.787295
[150]	valid_0's auc: 0.859756	valid_0's pr_auc: 0.790271
[200]	valid_0's auc: 0.861934	valid_0's pr_auc: 0.802815
[250]	valid_0's auc: 0.864637	valid_0's pr_auc: 0.80516
[300]	valid_0's auc: 0.865213	valid_0's pr_auc: 0.804284
[50]	valid_0's auc: 0.845986	valid_0's pr_auc: 0.778081
[100]	valid_0's auc: 0.848846	valid_0's pr_auc: 0.785118
[150]	valid_0's auc: 0.849107	valid_0's pr_auc: 0.782187
[200]	valid_0's auc: 0.849586	valid_0's pr_auc: 0.784552
[250]	valid_0's auc: 0.851715	valid_0's pr_auc: 0.788918
[300]	valid_0's auc: 0.854739	valid_0's pr_auc: 0.792077
[50]	valid_0's auc: 0.874084	valid_0's pr_auc: 0.82044
[100]	valid_0's auc: 0.876155	valid_0's pr_auc: 0.82223
[150]	valid_0's auc: 0.874498	valid_0's pr_auc: 0.82225
[200]	valid_0's auc: 0.876946	valid_0's pr_auc: 0.822465
[250]	valid_0's auc: 0.877924	valid_0's pr_auc: 0.82387
[300]	valid_0's auc: 0.877187	valid_0's 

# 4. 교차검증 점수 확인

In [67]:
# 위의 트레인 모델을 불러와서 사용 가능
import joblib
# joblib.dump( model, PATH + 'lgb.pkl')
# model = joblib.load( PATH + 'lgb.pkl')

In [156]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [157]:
sample_submssion = pd.read_csv(RAW_PATH+'sample_submission.csv')

In [158]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [160]:
sample_submssion.to_csv("test.csv", index = False)
# sample_submssion.to_csv("lb_score_0.84786.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.928659
1,30001,0.499147
2,30002,0.614533
3,30003,0.775865
4,30004,0.956908
...,...,...
14994,44994,0.366660
14995,44995,0.289906
14996,44996,0.754851
14997,44997,0.878986
